<a href="https://colab.research.google.com/github/inna-vitvinova/geoanalitika/blob/main/laba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install h3pandas --pre

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.8/931.8 kB 8.6 MB/s eta 0:00:00
  Created wheel for h3pandas: filename=h3pandas-0.2.6-py3-none-any.whl size=19688 sha256=f37bf8acc7128b58631e8dfe9914e564817d7c4126f5f4dd3bcaea26ae93a9aa
  Stored in directory: /root/.cache/pip/wheels/53/16/29/87a50ed96709a186f34192adccb57d691f2c5b6605bae71a3b
Successfully built h3pandas


In [2]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 905.1 kB/s eta 0:00:00


In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon

In [7]:
import pandas as pd
file = pd.read_csv('/content/ural-fed-district-latest-points.csv')
file

<ipython-input-7-cb0ac2c6aa15>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('/content/ural-fed-district-latest-points.csv')


,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags
0,27503892,Екатеринбург,NaN,NaN,NaN,NaN,NaN,city,NaN,"""addr:country""=>""RU"",""addr:region""=>""Свердловс..."
1,27504156,Челябинск,NaN,NaN,NaN,NaN,NaN,city,NaN,"""addr:country""=>""RU"",""addr:district""=>""Челябин..."
2,27505666,Тюмень,NaN,NaN,NaN,NaN,NaN,city,NaN,"""addr:country""=>""RU"",""addr:district""=>""городск..."
3,34506055,Нижний Тагил,NaN,NaN,NaN,NaN,NaN,city,NaN,"""addr:country""=>""RU"",""addr:district""=>""городск..."
4,128435293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""maxheight""=>""4.5"""
...,...,...,...,...,...,...,...,...,...,...
583091,11355895499,NaN,lift_gate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583092,11355895500,NaN,lift_gate,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583093,11355924469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""natural""=>""peak"""
583094,11356388483,урочище Механическое,NaN,NaN,NaN,NaN,NaN,locality,NaN,NaN


from matplotlib import pyplot as plt
file['osm_id'].plot(kind='hist', bins=20, title='osm_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['osm_id']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'osm_id'}, axis=1)
              .sort_values('osm_id', ascending=True))
  xs = counted['osm_id']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = file.sort_values('osm_id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('osm_id')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
file['osm_id'].plot(kind='line', figsize=(8, 4), title='osm_id')
plt.gca().spines[['top', 'right']].set_visible(False)

In [4]:
# отображаем 2 города
df = ox.geocode_to_gdf('Екатеринбург')
df2 = ox.geocode_to_gdf('Тюмень')

maps = []
m = folium.Map([df.centroid.y, df.centroid.x], tiles='cartodbpositron')
folium.GeoJson(df).add_to(m)
#folium.GeoJson(df.h3.polyfill_resample(9)).add_to(m) # не удалось применить библиотеку h3
folium.FitBounds([[df.bounds.miny[0], df.bounds.minx[0]],[df.bounds.maxy[0], df.bounds.maxx[0]]]).add_to(m)
m2 = folium.Map([df2.centroid.y, df2.centroid.x], tiles='cartodbpositron')
folium.GeoJson(df2).add_to(m2)
#folium.GeoJson(df2.h3.polyfill_resample(9)).add_to(m2)
folium.FitBounds([[df2.bounds.miny[0], df2.bounds.minx[0]],[df2.bounds.maxy[0], df2.bounds.maxx[0]]]).add_to(m2)
maps.append(m)
maps.append(m2)
for map in maps:
  display(map)

<ipython-input-4-135eb4765bd5>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map([df.centroid.y, df.centroid.x], tiles='cartodbpositron')
<ipython-input-4-135eb4765bd5>:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m2 = folium.Map([df2.centroid.y, df2.centroid.x], tiles='cartodbpositron')
